# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 3:04PM,249561,10,PRF-40441,Bio-PRF,Released
1,Oct 27 2022 3:04PM,249561,10,PRF-40451,Bio-PRF,Released
2,Oct 27 2022 3:04PM,249561,10,PRF-40464,Bio-PRF,Released
3,Oct 27 2022 3:04PM,249561,10,PRF-40467,Bio-PRF,Released
4,Oct 27 2022 2:59PM,249560,10,8700141,Yusen – 3D Matrix,Released
5,Oct 27 2022 2:58PM,249559,19,60018562,"GUSA Granules USA, Inc.",Released
6,Oct 27 2022 2:57PM,249558,10,0086122926,ISDIN Corporation,Released
7,Oct 27 2022 2:57PM,249558,10,0086122925,ISDIN Corporation,Released
8,Oct 27 2022 2:57PM,249558,10,0086122924,ISDIN Corporation,Released
9,Oct 27 2022 2:57PM,249558,10,0086123038,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,249557,Released,1
33,249558,Released,13
34,249559,Released,1
35,249560,Released,1
36,249561,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249557,NaN,NaN,1.0
249558,NaN,NaN,13.0
249559,NaN,NaN,1.0
249560,NaN,NaN,1.0
249561,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0.0,1.0,0.0
249472,26.0,14.0,1.0
249479,0.0,1.0,0.0
249485,0.0,1.0,0.0
249494,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0,1,0
249472,26,14,1
249479,0,1,0
249485,0,1,0
249494,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,0,1,0
1,249472,26,14,1
2,249479,0,1,0
3,249485,0,1,0
4,249494,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,,1,
1,249472,26,14,1
2,249479,,1,
3,249485,,1,
4,249494,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 3:04PM,249561,10,Bio-PRF
4,Oct 27 2022 2:59PM,249560,10,Yusen – 3D Matrix
5,Oct 27 2022 2:58PM,249559,19,"GUSA Granules USA, Inc."
6,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation
19,Oct 27 2022 2:56PM,249557,10,Yusen – 3D Matrix
20,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc."
21,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc."
22,Oct 27 2022 2:34PM,249552,10,Bio-PRF
23,Oct 27 2022 2:31PM,249551,10,Bio-PRF
24,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 27 2022 3:04PM,249561,10,Bio-PRF,,,4
1,Oct 27 2022 2:59PM,249560,10,Yusen – 3D Matrix,,,1
2,Oct 27 2022 2:58PM,249559,19,"GUSA Granules USA, Inc.",,,1
3,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,,,13
4,Oct 27 2022 2:56PM,249557,10,Yusen – 3D Matrix,,,1
5,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc.",,,1
6,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc.",,,1
7,Oct 27 2022 2:34PM,249552,10,Bio-PRF,,,1
8,Oct 27 2022 2:31PM,249551,10,Bio-PRF,,,1
9,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 3:04PM,249561,10,Bio-PRF,4,,
1,Oct 27 2022 2:59PM,249560,10,Yusen – 3D Matrix,1,,
2,Oct 27 2022 2:58PM,249559,19,"GUSA Granules USA, Inc.",1,,
3,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,13,,
4,Oct 27 2022 2:56PM,249557,10,Yusen – 3D Matrix,1,,
5,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc.",1,,
6,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc.",1,,
7,Oct 27 2022 2:34PM,249552,10,Bio-PRF,1,,
8,Oct 27 2022 2:31PM,249551,10,Bio-PRF,1,,
9,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 3:04PM,249561,10,Bio-PRF,4,,
1,Oct 27 2022 2:59PM,249560,10,Yusen – 3D Matrix,1,,
2,Oct 27 2022 2:58PM,249559,19,"GUSA Granules USA, Inc.",1,,
3,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,13,,
4,Oct 27 2022 2:56PM,249557,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 3:04PM,249561,10,Bio-PRF,4.0,NaN,NaN
1,Oct 27 2022 2:59PM,249560,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Oct 27 2022 2:58PM,249559,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,13.0,NaN,NaN
4,Oct 27 2022 2:56PM,249557,10,Yusen – 3D Matrix,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 3:04PM,249561,10,Bio-PRF,4.0,0.0,0.0
1,Oct 27 2022 2:59PM,249560,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Oct 27 2022 2:58PM,249559,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,13.0,0.0,0.0
4,Oct 27 2022 2:56PM,249557,10,Yusen – 3D Matrix,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1746885,22.0,1.0,0.0
15,1746598,108.0,15.0,27.0
16,1247582,3.0,2.0,0.0
19,748643,5.0,4.0,0.0
20,498979,0.0,2.0,0.0
21,1745580,6.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1746885,22.0,1.0,0.0
1,15,1746598,108.0,15.0,27.0
2,16,1247582,3.0,2.0,0.0
3,19,748643,5.0,4.0,0.0
4,20,498979,0.0,2.0,0.0
5,21,1745580,6.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,22.0,1.0,0.0
1,15,108.0,15.0,27.0
2,16,3.0,2.0,0.0
3,19,5.0,4.0,0.0
4,20,0.0,2.0,0.0
5,21,6.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,22.0
1,15,Released,108.0
2,16,Released,3.0
3,19,Released,5.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,27.0,0.0,0.0,0.0,0.0
Executing,1.0,15.0,2.0,4.0,2.0,1.0
Released,22.0,108.0,3.0,5.0,0.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,27.0,0.0,0.0,0.0,0.0
1,Executing,1.0,15.0,2.0,4.0,2.0,1.0
2,Released,22.0,108.0,3.0,5.0,0.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,27.0,0.0,0.0,0.0,0.0
1,Executing,1.0,15.0,2.0,4.0,2.0,1.0
2,Released,22.0,108.0,3.0,5.0,0.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()